In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torchvision
import argparse
from torch.autograd import Variable
import torch.utils.data as data

#from data import v2, v1, AnnotationTransform, VOCDetection, detection_collate, VOCroot, VOC_CLASSES
from data import v2, v1, detection_collate
from data import MSCOCODetection, COCOAnnotationTransform
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import numpy as np
import time
import sys

#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="1"

COMMON_CLASSES = ('person', 'bicycle', 'car',
                  'bus', 'train', 'boat',
                   'bird', 'cat', 'dog',
                   'horse', 'sheep', 'cow',
                   'bottle', 'chair', 'airplane',
                 'dining table', 'potted plant', 'tv',
                 'motorcycle', 'couch')

#hyperparameters

version = "v2"
basenet = "vgg16_reducedfc.pth"
jaccard_threshold=0.5
lr=1e-5
cwd = os.getcwd()
save_folder=cwd + "/MSCOCO14weights/"

voc_root= "/new_data/gpu/utkrsh/coco/" # location of the image root directory

annFile = "/new_data/gpu/utkrsh/coco/annotations/instances_train2014.json"
train_img = "/new_data/gpu/utkrsh/coco/images/train2014/"

cuda=False
resume = "./MSCOCO14weights/ssd300_COCO_8000.pth" # saved trained weights
start_iter=8001

if cuda and torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

if not os.path.exists(save_folder):
    os.mkdir(save_folder)

#train_sets = [('2007', 'trainval'), ('2012', 'trainval')]
# train_sets = 'train'
ssd_dim = 300  # only support 300 now
means = (104, 117, 123)  # imagenet mean values
num_classes = len(COMMON_CLASSES) + 1
batch_size = 32
accum_batch_size = 32
iter_size = accum_batch_size / batch_size
max_iter = 120000
weight_decay = 0.0005
stepvalues = (10000, 20000, 40000, 80000, 100000, 120000)
gamma = 0.1
momentum = 0.9

In [2]:
print('Loading Dataset...')
    
dataset = MSCOCODetection(image_root=train_img, ann_root=annFile,
                          transform=SSDAugmentation(ssd_dim, means), 
                          target_transform=COCOAnnotationTransform())
#dataset = MSCOCODetection(image_root=train_img, ann_root=annFile)
print("Dataset Loaded!")

data_loader = data.DataLoader(dataset, batch_size, num_workers=0,
                             shuffle=True, collate_fn=detection_collate,
                             pin_memory=cuda)

Loading Dataset...
loading annotations into memory...
Done (t=9.04s)
creating index...
index created!
Dataset Loaded!


In [3]:
batch = iter(data_loader)

In [15]:
_, targets = next(batch)

In [17]:
type(targets[0])

torch.FloatTensor

In [19]:
%time tmp = np.concatenate(list(map(lambda x: x.numpy(), targets)))

CPU times: user 115 µs, sys: 11 µs, total: 126 µs
Wall time: 139 µs


In [23]:
print(tmp[:10,:])

[[  1.14948645e-01   0.00000000e+00   7.07647085e-01   2.97035187e-01
    1.80000000e+01]
 [  9.35011685e-01   4.46646899e-01   9.65901613e-01   5.24569750e-01
    2.00000000e+00]
 [  2.32371137e-01   5.80506325e-02   8.24515462e-01   1.00000000e+00
    0.00000000e+00]
 [  3.54742259e-01   3.59746851e-02   6.48659766e-01   7.67746806e-01
    0.00000000e+00]
 [  0.00000000e+00   6.04405046e-01   3.70618552e-01   6.40101254e-01
    1.50000000e+01]
 [  7.75216520e-01   6.18987344e-02   1.00000000e+00   7.67822802e-01
    0.00000000e+00]
 [  3.58060002e-01   4.40870881e-01   6.71720028e-01   7.35495508e-01
    1.10000000e+01]
 [  1.35279998e-01   4.65855867e-01   3.34179997e-01   6.84414387e-01
    1.10000000e+01]
 [  2.34259993e-01   4.32702690e-01   2.90780008e-01   6.90750778e-01
    0.00000000e+00]
 [  3.00000003e-03   4.94384378e-01   1.94600001e-02   6.89879894e-01
    0.00000000e+00]]


In [24]:
type(tmp[2][3])

numpy.float32

In [3]:
def func(dataloader):
    batch = iter(data_loader)
    MIN, MAX = 100, -10
    i=1
    # for i in range(10):
    try:
        while True:
            _, targets = next(batch)
            # print(type(targets[0]))
            tmp = np.concatenate(list(map(lambda x: x.numpy(), targets)))
            # print(tmp[:10,:])
            xtmp = tmp[:,2]-tmp[:,0]
            ytmp = tmp[:,3]-tmp[:,1]
            area = xtmp*ytmp
            tmin = min(area)
            tmax = max(area)
            MIN = min(MIN,tmin)
            MAX = max(MAX,tmax)
            i=i+1
            if i % 20 == 0:
                print("batchs: ",i,"|| min: ",tmin," || max: ",tmax)
    except:
        print("Overall min: ",MIN," || max: ",MAX)
    finally:
        del batch

In [4]:
func(data_loader)

batchs:  20 || min:  1.74848e-05  || max:  1.0
batchs:  40 || min:  7.29635e-05  || max:  1.0
batchs:  60 || min:  0.000147988  || max:  1.0
batchs:  80 || min:  2.17252e-05  || max:  1.0
batchs:  100 || min:  5.79732e-05  || max:  1.0
batchs:  120 || min:  0.000111237  || max:  0.7559
batchs:  140 || min:  9.77514e-05  || max:  1.0
batchs:  160 || min:  0.000216419  || max:  0.997083
batchs:  180 || min:  6.60735e-05  || max:  1.0
batchs:  200 || min:  5.43283e-06  || max:  1.0
batchs:  220 || min:  2.24901e-05  || max:  1.0
batchs:  240 || min:  7.90025e-06  || max:  1.0
batchs:  260 || min:  6.38638e-05  || max:  1.0
batchs:  280 || min:  2.03464e-05  || max:  1.0
batchs:  300 || min:  6.42749e-05  || max:  0.973213
batchs:  320 || min:  1.63821e-05  || max:  1.0
batchs:  340 || min:  8.87936e-05  || max:  0.985437
batchs:  360 || min:  0.000171507  || max:  1.0
batchs:  380 || min:  3.89461e-05  || max:  0.849121
batchs:  400 || min:  5.11005e-05  || max:  1.0
batchs:  420 || min: 